# Import Libraries and Global Configuration

In [ ]:
import pandas as pd
import numpy as np
import os
from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.statistics import logrank_test
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# --- Global Configuration ---

# 1. Define file paths
BASE_DATA_PATH = "/your path/cardiomicscore/data/split_seed-250901/"
SCORES_PATH = "/your path/cardiomicscore/saved/results/Scores/OmicsNet/Final/"
RESULTS_SAVE_DIR = "/your path/cardiomicscore/saved/results/Survival_Analysis/"

# 2. Define outcomes and scores to be analyzed
OUTCOMES_TO_ANALYZE = ['cad', 'stroke', 'hf', 'af', 'pad', 'vte']
SCORES_TO_ANALYZE = ['prs', 'metscore', 'proscore']
SCORES_TO_STANDARDIZE = ['metscore', 'proscore'] # PRS is typically already standardized

# Ensure the results directory exists
os.makedirs(RESULTS_SAVE_DIR, exist_ok=True)

# Data Loading and Preparation Function

In [ ]:
def load_and_prepare_data(outcome: str):
    """
    Dynamically loads, merges, and preprocesses all necessary data for a specific outcome.
    
    Args:
        outcome (str): The name of the outcome to analyze (e.g., 'af', 'cad').

    Returns:
        pd.DataFrame: A DataFrame containing all relevant data. Returns an empty DataFrame if critical files are missing.
    """
    print(f"--- Loading and Preparing Data for Outcome: {outcome.upper()} ---")
    
    # 1. Load clinical data
    try:
        clinical_path = os.path.join(BASE_DATA_PATH, "X_external_test_PANEL.feather")
        main_df = pd.read_feather(clinical_path)
    except FileNotFoundError:
        print(f"ERROR: Clinical data file not found at {clinical_path}")
        return pd.DataFrame()

    # 2. Load and merge various scores from consolidated files
    score_map = {
        'metscore': 'Metabolomics',
        'proscore': 'Proteomics'
    }
    for score_name, score_type in score_map.items():
        try:
            # Filename is now generic: external_test_scores_{score_type}.csv
            score_file = f"external_test_scores_{score_type}.csv"
            score_path = os.path.join(SCORES_PATH, score_file)
            # Read the CSV and select the column corresponding to the current outcome
            score_df = pd.read_csv(score_path)[['eid', outcome]].rename(columns={outcome: score_name})
            main_df = pd.merge(main_df, score_df, on='eid', how='inner')
        except (FileNotFoundError, KeyError) as e:
            print(f"WARNING: Score file '{score_file}' or outcome column '{outcome}' not found. Skipping '{score_name}'. Error: {e}")

    # 3. Load and merge Genomics PRS score (dynamic column name)
    try:
        genomics_path = os.path.join(BASE_DATA_PATH, "X_external_test_Genomics.feather")
        prs_col_name = f'{outcome}_prs'
        prs_df = pd.read_feather(genomics_path)[['eid', prs_col_name]].rename(columns={prs_col_name: 'prs'})
        main_df = pd.merge(main_df, prs_df, on='eid', how='inner')
    except (FileNotFoundError, KeyError) as e:
        print(f"WARNING: Genomics file or PRS column '{prs_col_name}' not found. Skipping 'prs'. Error: {e}")

    # 4. Load and merge outcome data from consolidated files (dynamic column names)
    try:
        y_path = os.path.join(BASE_DATA_PATH, "y_external_test.feather")
        e_path = os.path.join(BASE_DATA_PATH, "e_external_test.feather")
        
        duration_col = f'bl2{outcome}_yrs'
        event_col = outcome
        
        y_df = pd.read_feather(y_path)[['eid', duration_col]].rename(columns={duration_col: 'duration'})
        e_df = pd.read_feather(e_path)[['eid', event_col]].rename(columns={event_col: 'event'})
        
        main_df = pd.merge(main_df, y_df, on='eid', how='inner')
        main_df = pd.merge(main_df, e_df, on='eid', how='inner')
    except (FileNotFoundError, KeyError) as e:
        print(f"WARNING: Outcome files for '{outcome}' not found or column missing. Error: {e}")
        return pd.DataFrame()
        
    # 5. Standardize scores
    actual_scores_to_standardize = [col for col in SCORES_TO_STANDARDIZE if col in main_df.columns]
    if actual_scores_to_standardize:
        scaler = StandardScaler()
        main_df[actual_scores_to_standardize] = scaler.fit_transform(main_df[actual_scores_to_standardize])
        
    print(f"--- Data for {outcome.upper()} is ready. Shape: {main_df.shape} ---\n")
    return main_df

# Analysis Functions

In [ ]:
def calculate_event_rate_by_percentile(df: pd.DataFrame, score_columns: list):
    """Calculates the event rate for each percentile of the given scores."""
    all_percentile_data = []
    for score_col in score_columns:
        if score_col not in df.columns:
            continue
        
        score_series = df[score_col]
        for j in range(100):
            lower_bound = score_series.quantile(j / 100.0)
            upper_bound = score_series.quantile((j + 1) / 100.0)
            
            if j == 99:
                subset = df[(score_series >= lower_bound) & (score_series <= upper_bound)]
            else:
                subset = df[(score_series >= lower_bound) & (score_series < upper_bound)]

            n_total = len(subset)
            n_events = subset['event'].sum() if n_total > 0 else 0
            event_rate = n_events / n_total if n_total > 0 else 0.0
            
            all_percentile_data.append({
                'score_type': score_col,
                'percentile': j + 1,
                'event_rate': event_rate,
                'n_total': n_total,
                'n_events': n_events
            })

    return pd.DataFrame(all_percentile_data)

def analyze_km_and_logrank(df: pd.DataFrame, score_column: str, outcome: str):
    """Performs Kaplan-Meier analysis and log-rank test for a given score."""
    if score_column not in df.columns:
        return None, None

    # Stratify by score tertiles
    try:
        low_cutoff = df[score_column].quantile(1/3)
        high_cutoff = df[score_column].quantile(2/3)
        conditions = [
            df[score_column] <= low_cutoff,
            (df[score_column] > low_cutoff) & (df[score_column] <= high_cutoff),
        ]
        choices = ["Low", "Medium"]
        group_col = f'{score_column}_group'
        df[group_col] = np.select(conditions, choices, default="High")
        
        if df[group_col].nunique() < 2:
            raise ValueError("Stratification resulted in fewer than 2 groups.")
    except ValueError as e:
        print(f"Could not stratify {score_column} for outcome {outcome}. Error: {e}.")
        return None, None
        
    # KM plotting and data preparation
    plt.figure(figsize=(8, 6))
    ax = plt.subplot(111)
    kmf = KaplanMeierFitter()
    all_survival_data = []

    for name in sorted(df[group_col].unique()):
        grouped_df = df[df[group_col] == name]
        kmf.fit(grouped_df['duration'], grouped_df['event'], label=f'{name} Risk')
        kmf.plot_survival_function(ax=ax)
        
        survival_df = kmf.confidence_interval_survival_function_.copy()
        survival_df['time'] = kmf.survival_function_.index
        survival_df['survival_prob'] = kmf.survival_function_.iloc[:, 0]
        survival_df['group'] = name
        survival_df['score_type'] = score_column
        survival_df = survival_df.rename(columns={
            f'{kmf.label}_lower_0.95': 'ci_lower',
            f'{kmf.label}_upper_0.95': 'ci_upper'
        })
        all_survival_data.append(survival_df)
        
    plt.title(f"Kaplan-Meier Curves for {outcome.upper()} by {score_column.upper()} Group")
    plt.xlabel("Time (Years)")
    plt.ylabel("Survival Probability")
    plt.ylim(0.8, 1.0)
    plt.grid(True)
    plt.legend()
    plt.show()

    km_results_df = pd.concat(all_survival_data, ignore_index=True)
        
    # Log-Rank Test
    groups = df[group_col]
    group_labels = sorted(groups.unique())
    logrank_results = None
    if len(group_labels) == 3:
        results_logrank = logrank_test(
            durations_A=df.loc[groups == group_labels[0], 'duration'], event_observed_A=df.loc[groups == group_labels[0], 'event'],
            durations_B=df.loc[groups == group_labels[1], 'duration'], event_observed_B=df.loc[groups == group_labels[1], 'event'],
            durations_C=df.loc[groups == group_labels[2], 'duration'], event_observed_C=df.loc[groups == group_labels[2], 'event']
        )
        p_value = results_logrank.p_value
        if p_value < 0.001:
            p_value_formatted = f"{p_value:.2e}"  # Scientific notation
        else:
            p_value_formatted = f"{p_value:.3f}"  # 3 decimal places
        logrank_results = pd.DataFrame({
            'test_statistic': [results_logrank.test_statistic],
            'p': [p_value_formatted],
            '-log2(p)': [-np.log2(p_value) if p_value > 0 else np.inf]
        })
        logrank_results['score_type'] = score_column
        
    return km_results_df, logrank_results

def analyze_hazard_ratios(df: pd.DataFrame, score_column: str, all_clinical_vars: list):
    """
    Calculates and saves Hazard Ratios for a given score, analyzed as both a
    continuous variable and a categorical (tertile-based) variable.
    Includes printing of model details.
    """
    print(f"===== Analyzing Hazard Ratios for: {score_column.upper()} =====")

    if score_column not in df.columns:
        print(f"Score column '{score_column}' not found. Skipping analysis.\n")
        return None, None

    # --- 1. Analysis with the Score as a CONTINUOUS Variable ---
    print(f"\n--- Analyzing '{score_column}' as a Continuous Variable ---")
    continuous_results = []
    cph_continuous = CoxPHFitter(penalizer=0.03)

    # Unadjusted HR
    try:
        df_unadj = df[[score_column, 'duration', 'event']].dropna()
        print(f"Unadjusted model shape: {df_unadj.shape}")
        cph_continuous.fit(df_unadj, 'duration', 'event', formula=f"{score_column}")
        hr_unadj = cph_continuous.summary.loc[[score_column]].copy()
        hr_unadj['model'] = 'Unadjusted'
        continuous_results.append(hr_unadj)
    except Exception as e:
        print(f"Failed to fit unadjusted continuous model for {score_column}: {e}")

    # Adjusted HR for Age & Sex
    try:
        adj_vars_age_sex = ['age', 'male_1.0']
        df_adj_as = df[[score_column] + adj_vars_age_sex + ['duration', 'event']].dropna()
        print(f"Age+Sex adjustment variables: {adj_vars_age_sex}")
        print(f"Age+Sex adjusted model shape: {df_adj_as.shape}")
        cph_continuous.fit(df_adj_as, 'duration', 'event')
        hr_adj_as = cph_continuous.summary.loc[[score_column]].copy()
        hr_adj_as['model'] = 'Adjusted (Age+Sex)'
        continuous_results.append(hr_adj_as)
    except Exception as e:
        print(f"Failed to fit partially adjusted continuous model for {score_column}: {e}")

    # Adjusted HR for all clinical variables
    try:
        other_clinical_vars = [v for v in all_clinical_vars if v != score_column]
        df_adj_all = df[[score_column] + other_clinical_vars + ['duration', 'event']].dropna()
        print(f"All Clinical adjustment variables: {other_clinical_vars}")
        print(f"All Clinical adjusted model shape: {df_adj_all.shape}")
        cph_continuous.fit(df_adj_all, 'duration', 'event')
        hr_adj_all = cph_continuous.summary.loc[[score_column]].copy()
        hr_adj_all['model'] = 'Adjusted (All Clinical)'
        continuous_results.append(hr_adj_all)
    except Exception as e:
        print(f"Failed to fit fully adjusted continuous model for {score_column}: {e}")

    continuous_results_df = pd.concat(continuous_results).reset_index().rename(columns={'index': 'covariate'})
    continuous_results_df['score_type'] = score_column

    # --- 2. Analysis with the Score as a CATEGORICAL Variable (Tertiles) ---
    print(f"\n--- Analyzing '{score_column}' as a Categorical (Tertile) Variable ---")
    categorical_results = []
    cph_categorical = CoxPHFitter(penalizer=0.01)

    # Create the categorical group column
    try:
        low_cutoff = df[score_column].quantile(1/3)
        high_cutoff = df[score_column].quantile(2/3)
        risk_categories = ["Low", "Medium", "High"]
        group_col = f'{score_column}_group'
        df[group_col] = pd.cut(df[score_column], bins=[-np.inf, low_cutoff, high_cutoff, np.inf], labels=risk_categories)
        df[group_col] = pd.Categorical(df[group_col], categories=risk_categories, ordered=True)
    except Exception as e:
        print(f"Could not stratify {score_column}. Skipping categorical analysis. Error: {e}")
        return continuous_results_df, None
    
    # Define the score group columns to keep in the final output
    score_group_columns = [f'{score_column}_group_Medium', f'{score_column}_group_High']

    # Unadjusted HR for categories
    try:
        df_cat_unadj = pd.get_dummies(df[[group_col, 'duration', 'event']].dropna(), columns=[group_col], drop_first=True)
        cph_categorical.fit(df_cat_unadj, 'duration', 'event')
        summary_unadj = cph_categorical.summary.loc[score_group_columns].copy()
        summary_unadj['model'] = 'Unadjusted'
        categorical_results.append(summary_unadj)
    except Exception as e:
        print(f"Failed to fit unadjusted categorical model for {score_column}: {e}")

    # Adjusted HR for Age & Sex for categories
    try:
        df_cat_adj_as = pd.get_dummies(df[[group_col] + adj_vars_age_sex + ['duration', 'event']].dropna(), columns=[group_col], drop_first=True)
        cph_categorical.fit(df_cat_adj_as, 'duration', 'event')
        summary_adj_as = cph_categorical.summary.loc[score_group_columns].copy()
        summary_adj_as['model'] = 'Adjusted (Age+Sex)'
        categorical_results.append(summary_adj_as)
    except Exception as e:
        print(f"Failed to fit partially adjusted categorical model for {score_column}: {e}")

    # Adjusted HR for all clinical variables for categories
    try:
        df_cat_adj_all = pd.get_dummies(df[[group_col] + all_clinical_vars + ['duration', 'event']].dropna(), columns=[group_col], drop_first=True)
        cph_categorical.fit(df_cat_adj_all, 'duration', 'event')
        summary_adj_all = cph_categorical.summary.loc[score_group_columns].copy()
        summary_adj_all['model'] = 'Adjusted (All Clinical)'
        categorical_results.append(summary_adj_all)
    except Exception as e:
        print(f"Failed to fit fully adjusted categorical model for {score_column}: {e}")

    categorical_results_df = pd.concat(categorical_results).reset_index().rename(columns={'index': 'covariate'})
    categorical_results_df['score_type'] = score_column

    print(f"===== Finished Hazard Ratio Analysis for: {score_column.upper()} =====\n")
    
    return continuous_results_df, categorical_results_df

# Main Execution Loop

In [ ]:
# --- Main Execution Loop ---

# Initialize lists to store results from all outcomes
all_event_rates_list = []
all_km_data_list = []
all_logrank_list = []
all_hr_continuous_list = []
all_hr_categorical_list = []

# Get the list of all clinical variables once
try:
    clinical_path = os.path.join(BASE_DATA_PATH, "X_external_test_PANEL.feather")
    base_clinical_vars = [col for col in pd.read_feather(clinical_path).columns if col != 'eid']
except FileNotFoundError:
    print("ERROR: X_external_test_Clinical.feather not found. Using minimal adjustment vars.")
    base_clinical_vars = ['age', 'male_1.0']

# Loop through each outcome
for outcome in OUTCOMES_TO_ANALYZE:
    master_df = load_and_prepare_data(outcome)
    
    if master_df.empty:
        print(f"Skipping analysis for outcome {outcome} due to missing data.")
        continue
    
    # --- 1. Event Rate Analysis ---
    print(f"Calculating event rates for {outcome.upper()}...")
    event_rate_df = calculate_event_rate_by_percentile(master_df, SCORES_TO_ANALYZE)
    event_rate_df['outcome'] = outcome
    all_event_rates_list.append(event_rate_df)
    
    # --- 2. KM and Log-rank Analysis ---
    print(f"Performing Kaplan-Meier analysis for {outcome.upper()}...")
    for score in SCORES_TO_ANALYZE:
        km_df, logrank_df = analyze_km_and_logrank(master_df.copy(), score, outcome)
        if km_df is not None:
            km_df['outcome'] = outcome
            all_km_data_list.append(km_df)
        if logrank_df is not None:
            logrank_df['outcome'] = outcome
            all_logrank_list.append(logrank_df)
            
    # --- 3. Hazard Ratio Analysis ---
    print(f"Calculating Hazard Ratios for {outcome.upper()}...")
    for score in SCORES_TO_ANALYZE:
        adjustment_vars = [var for var in base_clinical_vars if var != score]
        hr_cont_df, hr_cat_df = analyze_hazard_ratios(master_df.copy(), score, adjustment_vars)
        if hr_cont_df is not None:
            hr_cont_df['outcome'] = outcome
            all_hr_continuous_list.append(hr_cont_df)
        if hr_cat_df is not None:
            hr_cat_df['outcome'] = outcome
            all_hr_categorical_list.append(hr_cat_df)

print("\n--- All analyses complete. Aggregating and saving results. ---")

# Result Aggregation and Saving

In [ ]:
# 1. Observed Event Rates
if all_event_rates_list:
    final_event_rates = pd.concat(all_event_rates_list, ignore_index=True)
    save_path = os.path.join(RESULTS_SAVE_DIR, "summary_observed_event_rate.csv")
    final_event_rates.to_csv(save_path, index=False, float_format='%.6f')
    print(f"\nSaved combined event rate data to: {save_path}")
    print("Event Rate Data Preview:")
    print(final_event_rates.head())

# 2. KM Data
if all_km_data_list:
    final_km_data = pd.concat(all_km_data_list, ignore_index=True)
    save_path = os.path.join(RESULTS_SAVE_DIR, "summary_survival_km_data.csv")
    final_km_data.to_csv(save_path, index=False, float_format='%.6f')
    print(f"\nSaved combined KM survival data to: {save_path}")

# 3. Log-rank Results
if all_logrank_list:
    final_logrank_data = pd.concat(all_logrank_list, ignore_index=True)
    save_path = os.path.join(RESULTS_SAVE_DIR, "summary_logrank_test_results.csv")
    final_logrank_data.to_csv(save_path, index=False, float_format='%.4f')
    print(f"\nSaved combined Log-rank test results to: {save_path}")

# 4. Continuous HR Results
if all_hr_continuous_list:
    final_hr_continuous = pd.concat(all_hr_continuous_list, ignore_index=True)
    save_path = os.path.join(RESULTS_SAVE_DIR, "summary_hazard_ratio_continuous.csv")
    final_hr_continuous.to_csv(save_path, index=False, float_format='%.4f')
    print(f"\nSaved combined continuous HR results to: {save_path}")

# 5. Categorical HR Results
if all_hr_categorical_list:
    final_hr_categorical = pd.concat(all_hr_categorical_list, ignore_index=True)
    save_path = os.path.join(RESULTS_SAVE_DIR, "summary_hazard_ratio_categorical.csv")
    final_hr_categorical.to_csv(save_path, index=False, float_format='%.4f')
    print(f"\nSaved combined categorical HR results to: {save_path}")

print("\n--- Script finished successfully! ---")